# Investigating the Practicality of Adversarial Evasion Attacks on Network Intrusion Detection (CIDDS-001)

## Libraries import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import copy
import time as time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.feature_selection import chi2

#!pip install adversarial-robustness-toolbox >/dev/null
import os,sys
sys.path.append(os.path.dirname('../adversarial-robustness-toolbox/'))
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, DeepFool, CarliniL2Method, CarliniLInfMethod
from art.classifiers import PyTorchClassifier

%matplotlib inline

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

## CIDDS Data Preprocessing

### Dowloading and importing the dataset

In [3]:
#Downloading and extracting the dataset if it doesn't exist
!if [ ! -d "./CIDDS-001" ]; then wget https://www.hs-coburg.de/fileadmin/hscoburg/WISENT-CIDDS-001.zip; unzip WISENT-CIDDS-001.zip; fi 
#Importing the training and testing datasets from .CSV to Pandas DataFrames
df_training = pd.read_csv('./CIDDS-001/traffic/OpenStack/CIDDS-001-internal-week1.csv')
df_testing = pd.read_csv('./CIDDS-001/traffic/OpenStack/CIDDS-001-internal-week2.csv')
# Stack the training and test sets
data = pd.concat([df_training, df_testing], axis=0)

#data = data.iloc[:100000, :]
#data

--2021-05-26 10:10:00--  https://www.hs-coburg.de/fileadmin/hscoburg/WISENT-CIDDS-001.zip
Resolving www.hs-coburg.de (www.hs-coburg.de)... 192.129.27.193
Connecting to www.hs-coburg.de (www.hs-coburg.de)|192.129.27.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381232494 (364M) [application/zip]
Saving to: ‘WISENT-CIDDS-001.zip’

WISENT-CIDDS-001.zi 100%[===================>] 363.57M   576KB/s    in 11m 1s  

2021-05-26 10:21:01 (564 KB/s) - ‘WISENT-CIDDS-001.zip’ saved [381232494/381232494]

Archive:  WISENT-CIDDS-001.zip
   creating: CIDDS-001/
   creating: CIDDS-001/attack_logs/
  inflating: CIDDS-001/attack_logs/attack_logs_extern.csv  
  inflating: CIDDS-001/attack_logs/attack_logs_intern.csv  
   creating: CIDDS-001/client_confs/
  inflating: CIDDS-001/client_confs/192.168.200.4.conf  
  inflating: CIDDS-001/client_confs/192.168.200.5.conf  
  inflating: CIDDS-001/client_confs/192.168.200.8.conf  
  inflating: CIDDS-001/client_confs/192.168.200.9.co

/home/amine/anaconda3/envs/advids/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Removing unused features

In [ ]:
data.drop(['Src IP Addr','Dst IP Addr', 'attackType', 'attackID', 'attackDescription', 'Flows'], inplace=True, axis=1 )

### Creating a Timestamp feature

In [ ]:
data['Datetime'] = pd.to_datetime(data['Date first seen'], format='%Y-%m-%d %H:%M:%S.%f')
data['Timestamp'] = data.Datetime.values.astype(np.int64) // 10 ** 6
#data['Weekday'] = data.Datetime.dt.weekday
data.drop(['Date first seen','Datetime'], inplace=True, axis=1 )

### Creating source and destination port class features

In [ ]:
data['Src Pt:WellKnown'] = (data['Src Pt'] < 1024).astype(float)
#data['Src Pt:Registered'] = (data['Src Pt'] >= 1024 and data['Src Pt'] < 49152).astype(int) # didn't work
data['Src Pt:Registered'] = ((data['Src Pt'] >= 1024).astype(float) * (data['Src Pt'] < 49152).astype(float)).astype(float)
data['Src Pt:Dynamic'] = (data['Src Pt'] >= 49152).astype(float)
data.drop(['Src Pt'], inplace=True, axis=1)

data['Dst Pt:WellKnown'] = (data['Dst Pt'] < 1024).astype(float)
#data['Dst Pt:Registered'] = (data['Dst Pt'] >= 1024 and data['Dst Pt'] < 49152).astype(int) # didn't work
data['Dst Pt:Registered'] = ((data['Dst Pt'] >= 1024).astype(float) * (data['Dst Pt'] < 49152).astype(float)).astype(float)
data['Dst Pt:Dynamic'] = (data['Dst Pt'] >= 49152).astype(float)
data.drop(['Dst Pt'], inplace=True, axis=1)


### Transforming the problem into binary clasification

In [ ]:
# Labels as normal or attack (0/1)
data['Label'] = (data['class'] != 'normal').astype(float)
data.drop(['class'], inplace=True, axis=1 )

### One Hot Encoding the categorical features

In [ ]:
#for i in ['Weekday', 'Proto', 'Flags']:
for i in ['Proto', 'Flags']:
    # Create the One Hot Encode DataFrame
    print(i)
    dum = pd.get_dummies(data[i], dtype=float)
    # Insert into the dataset DataFrame by Series
    for column_name in list(dum.columns):
        new_column = str(i)+':'+str(column_name)
        data.insert(1,new_column , dum[column_name])
    # Drop the old attribute's column
    data.drop(i, inplace=True, axis=1)

### Converting Bytes from str to float

In [ ]:
# Bytes from str to float
data['Bytes'].astype(str)
data['Bytes'] = data['Bytes'].apply(lambda x: x.replace(' ', '') if type(x) == str else x)
data['Bytes'] = data['Bytes'].apply(lambda x: (float(x[:-1])* 10**20) if type(x) == str and x[-1] == 'M' else float(x))

### Spliting the training and test set 

In [ ]:
# Split training and test sets
df_training = data[:df_training.shape[0]]    
df_testing = data[df_training.shape[0]:]

### Normalizing the data using Min-Max

In [ ]:
# Min-Max normalization on the non binary features
for i in ['Duration', 'Packets', 'Bytes', 'Tos', 'Timestamp']:
    # The min and max are only computed from the training set
    min = df_training[i].min()
    max = df_training[i].max()
    df_training[i] = ((df_training[i] - min) / (max - min)) 
    df_testing[i] = ((df_testing[i] - min) / (max - min)) 

### Convert the training and testing set into NumPy array

In [ ]:
# Get NumPy arrays from DataFrames
nd_training = df_training.values
nd_testing = df_testing.values

### Extracting the labels and making copies

In [ ]:
# Separating arguments (x) from lables (y)
x_train = nd_training[:, :-1]
y_train = nd_training[:, -1]
x_test = nd_testing[:, :-1]
y_test = nd_testing[:, -1]

## Neural Network Model Training

### Convert the training and testing set into PyTorch tensors

In [ ]:
# Convert from numpy array to torch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

### Define a neural network with 2 ReLU hidden layers and a Softmax output

In [ ]:
class Network(nn.Module):
    ''' A basic neural network model '''
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()         #python2 : super(MLP, self).__init__()
        #defining the network's operations
        self.fc1 = nn.Linear(input_size, hidden_size[0])
        self.fc2 = nn.Linear(hidden_size[0], hidden_size[1])
        self.fc3 = nn.Linear(hidden_size[1], output_size)

    def forward(self, x, softmax=False): 
        a = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x.float())))))
        if softmax:
            y_pred = F.softmax(a, dim=1)
        else:
            y_pred = a

        return y_pred

### Define a function to compute the accuracy of the prediction

In [ ]:
def evaluate_predictions(predictions, real):
    ''' Evaluates the accuracy of the predictions'''
    n_correct = torch.eq(predictions, real).sum().item()
    accuracy = n_correct / len(predictions) * 100
    return accuracy

### Define a function that prints the models perfomance metrics

In [ ]:
def stat_model(model, x_train, y_train, x_test, y_test):
    ''' Prints statistics about the model performances on the dataset'''
    _, predictions_train = model(x_train, softmax=True).max(dim=1)
    #_, predictions_train = model(x_train).max(dim=1)
    accuracy_train = evaluate_predictions(predictions=predictions_train.long(), real=y_train)

    _, predictions_test = model(x_test, softmax=True).max(dim=1)
    #_, predictions_test = model(x_test).max(dim=1)
    accuracy_test = evaluate_predictions(predictions=predictions_test.long(), real=y_test)
    
    print("Final Training Accuracy: {0:.4f}%\nFinal Testing Accuracy : {1:.4f}%"
          .format(accuracy_train, accuracy_test))
    # Move the tensors back to CPU
    label_test_final = y_test.cpu().numpy()
    predictions_test_final = predictions_test.cpu().numpy()
    report = classification_report(label_test_final, predictions_test_final)
    print("Classification Report :")
    print(report)

### Train the model

In [ ]:
# Initialising the model
input_size=x_train.shape[1]
hidden_size=[16,16]
output_size=2
model = Network(input_size, hidden_size, output_size)

# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training on : {}".format(device))

# Transfering model and data to GPU
model = model.to(device)
x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

# Setting the Loss function and Adam learning rate
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)

# Variables to store the best performences (weights and accuracy)
best_model_weights = copy.deepcopy(model.state_dict())
best_accuracy = 0.0

# DataFrame for the learning curve plot
trace = pd.DataFrame(columns=['epoch', 'train_acc', 'test_acc'])
# Iterrating on the dataset
since = time.time()
for epoch in range(1000+1):
    # Forward pass
    y_pred = model(x_train) 
    # torch.max(dim=1) returns the maximum value of each line AND its index
    _, predictions = y_pred.max(dim=1)
    # Compute accuracy
    accuracy_train = evaluate_predictions(predictions=predictions.long(), real=y_train)
    # Compute loss
    loss = criterion(y_pred, y_train)

    # Testing model on the test set
    if epoch%10 == 0:
        _, predictions_test = model(x_test, softmax=True).max(dim=1)
        accuracy_test = evaluate_predictions(predictions=predictions_test.long(), real=y_test)
        # Keep track of the accuracies for the learning curve
        trace = trace.append([{'epoch':epoch,
                                'train_acc':accuracy_train,
                                'test_acc':accuracy_test}])
        # Save the best model's accuracy and parameters
        if accuracy_test > best_accuracy:
            best_accuracy = accuracy_test
            best_model_weights = copy.deepcopy(model.state_dict())
        # Displap statistics
        if epoch%100 == 0:
            time_elapsed = time.time() - since
            print("epoch: {0:4d} | loss: {1:.4f} | Train accuracy: {2:.4f}% | Test accuracy: {3:.4f}% [{4:.4f}%] | Running for : {5:.0f}m {6:.0f}s"
                  .format(epoch,
                          loss,
                          accuracy_train,
                          accuracy_test,
                          best_accuracy,
                          time_elapsed // 60,
                          time_elapsed % 60))

    # Zero all gradients
    optimizer.zero_grad()
    # Backward pass
    loss.backward()
    # Update weights
    optimizer.step()

# Compute the training time
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

### Display the learning curve and the model performances

In [ ]:
# Draw the learning curve
plt.figure(figsize=(20, 10))
plt.scatter(data=trace, x='epoch', y='train_acc', c="b", s=5)
plt.scatter(data=trace, x='epoch', y='test_acc', c="r", s=5)
plt.plot(trace['epoch'], trace['train_acc'], c="b")
plt.plot(trace['epoch'], trace['test_acc'], c="r")
plt.ylim((0, 100))
plt.yticks(np.arange(0, 100, 5))
plt.grid()
plt.legend()
plt.savefig("learning_curve.eps", format="eps", bbox_inches='tight')

# Loading the best weights and displaying the best model's performances
model.load_state_dict(best_model_weights)
stat_model(model, x_train, y_train, x_test, y_test)

### Saving/Loading the model

In [ ]:
#torch.save(model.state_dict(), "./model-cidds.pytorch")
model.load_state_dict(torch.load("./model-cidds.pytorch"))

## Adversarial Attacks

### Define a table for statistics

In [ ]:
adv_feat_stats = pd.DataFrame(index=df_training.columns[:-1])

adv_results = pd.DataFrame(index=['Accuracy', 
                                  'Mean perturbed features   [Mean L0]', 
                                  'Max perturbed features    [Max  L0]', 
                                  'Mean Euclidiant distance  [Mean L2]', 
                                  'Max Euclidiant distance   [Max  L2]', 
                                  'Mean Maximum perturbation [Mean Li]', 
                                  'Max Maximum perturbation  [Max  Li]'])

adv_inv = pd.DataFrame(index=['Invalid value range',
                              'Invalid binary values',
                              'Invalid class belonging'])

### Define a function to compute Lp norms

In [ ]:
def adv_norms(x_test_cpu, adversarial_examples_cpu):
    mean_l0 = np.mean(np.sum(x_test_cpu != adversarial_examples_cpu, axis=1))
    max_l0 = np.max(np.sum(x_test_cpu != adversarial_examples_cpu, axis=1))
    mean_l2 = np.mean(np.sum(np.power(x_test_cpu - adversarial_examples_cpu, 2), axis=1, keepdims=True))
    max_l2 = np.max(np.sum(np.power(x_test_cpu - adversarial_examples_cpu, 2), axis=1, keepdims=True))
    mean_li = np.mean(np.max(np.abs(x_test_cpu - adversarial_examples_cpu), axis=1, keepdims=True))
    max_li = np.max(np.max(np.abs(x_test_cpu - adversarial_examples_cpu), axis=1, keepdims=True))
    return [mean_l0, max_l0, mean_l2, max_l2, mean_li, max_li]

### Define a function to check the invalidation criteria

In [ ]:
def adv_criteria(x_test_cpu, adversarial_examples_cpu):
    # Verify value ranges
    min = x_test_cpu.min(axis=1, keepdims=True)
    max = x_test_cpu.max(axis=1, keepdims=True)
    adv_range = (adversarial_examples_cpu < min) | (adversarial_examples_cpu > max)
    adv_range = adv_range.any(axis=1, keepdims=True)
    adv_range = adv_range.sum(axis=0)
    #print("proportion of out-of-range values : {:.2f}% | {}/{}".format(adv[0]*100/x_test.shape[0], adv[0], x_test.shape[0]))

    # Binary values
    binary_feat_ind = list(range(1,25)) + list(range(29,35))
    adv_bin = adversarial_examples_cpu[:, binary_feat_ind]
    adv_bin = (adv_bin != 1) & ( adv_bin != 0)
    adv_bin = adv_bin.any(axis=1, keepdims=True)
    adv_bin = adv_bin.sum(axis=0)
    #print("proportion of non-binary values : {:.2f}% | {}/{}".format(adv[0]*100/x_test.shape[0], adv[0], x_test.shape[0]))

    # Multi class
    # Flag
    adv1 = adversarial_examples_cpu[:, 1:21] != 0
    adv1 = adv1.astype(int).sum(axis=1, keepdims=True) != 1
    adv1 = adv1.sum(axis=1, keepdims=True)

    # Proto
    adv2 = adversarial_examples_cpu[:, 21:25] != 0
    adv2 = adv2.astype(int).sum(axis=1, keepdims=True) != 1
    adv2 = adv2.sum(axis=1, keepdims=True)

    # Src Pt
    adv3 = adversarial_examples_cpu[:, 29:32] != 0
    adv3 = adv3.astype(int).sum(axis=1, keepdims=True) != 1
    adv3 = adv3.sum(axis=1, keepdims=True)
    
    # Dst Pt
    adv4 = adversarial_examples_cpu[:, 32:35] != 0
    adv4 = adv3.astype(int).sum(axis=1, keepdims=True) != 1
    adv4 = adv3.sum(axis=1, keepdims=True)

    adv_cat = adv1 | adv2 | adv3 | adv4
    adv_cat = adv_cat.sum(axis=0)
    #print("proportion of multiple category values : {:.2f}% | {}/{}".format(adv[0]*100/x_test.shape[0], adv[0], x_test.shape[0]))

    return [adv_range[0]*100/x_test.shape[0], adv_bin[0]*100/x_test.shape[0], adv_cat[0]*100/x_test.shape[0]]

### Extracte the attacks samples and copy them in the device

In [ ]:
positive_examples = df_testing[df_testing['Label'] == 1].values
x_test = torch.from_numpy((positive_examples[:, :-1])).float()
y_test = torch.from_numpy((positive_examples[:, -1])).float()
x_test = x_test.to(device)
y_test = y_test.to(device)

### Define the model in ART

In [ ]:
# Applying the PyTorch wrapper
classifier = PyTorchClassifier(model=model, loss=criterion, optimizer=optimizer, input_shape=input_size, nb_classes=output_size)

### Clean Data
The model performance on untouched attack samples.

In [ ]:
_, predictions_clean = model(x_test, softmax=True).max(dim=1)
accuracy_clean = evaluate_predictions(predictions=predictions_clean.long(), real=y_test)

attack='Clean'

# Exporting the clean examples in a .csv file
pd.DataFrame(np.hstack((x_test.cpu().numpy(),y_test.cpu().numpy().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("clean_examples.csv")

x_test_cpu = np.array(x_test.cpu())
adv_results[attack] = [accuracy_clean] + adv_norms(x_test_cpu, x_test_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, x_test_cpu)

print(adv_results[attack])
print(adv_inv[attack])

### Fast Gradient Sign Method
*Goodfellow et al. (2015) "Explaining and Harnessing Adversarial Examplse"*

In [ ]:
# Creating the adversarial examples crafter
adversarial_crafter = FastGradientMethod(classifier,
                                         norm=np.inf,
                                         eps=0.1,
                                         targeted=False,
                                         num_random_init=0,
                                         batch_size=128,
                                         )
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=x_test.cpu())

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate(predictions=predictions_adv.long(), real=y_test)
attack = 'FGSM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_FGSM.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

### Basic Iterative Method

*Kurakin et al. (2016) "Adversarial examples in the physical world"*


In [ ]:
# Creating the adversarial examples crafter
adversarial_crafter = BasicIterativeMethod(classifier, 
                                           eps=0.1, 
                                           eps_step=0.001,
                                           max_iter=100, 
                                           targeted=False, 
                                           batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=x_test.cpu())

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate(predictions=predictions_adv.long(), real=y_test)
attack = 'BIM'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_BIM.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

### DeepFool
*Moosavi-Dezfooli et al (2016) "DeepFool: a simple and accurate method to fool deep neural networks"*

In [ ]:
# Creating the adversarial examples crafter
adversarial_crafter = DeepFool(classifier, 
                               max_iter=100, 
                               epsilon=1e-6, 
                               nb_grads=10, 
                               batch_size=128)

# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=x_test.cpu())

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate(predictions=predictions_adv.long(), real=y_test)
attack = 'DF'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)            
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_DF.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 10e-6).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

### Carlini & Wagner L2 Attack
*Carlini et al. (2017) "Towards Evaluating the Robustness of Neural Networks"*

In [ ]:
# Creating the adversarial examples crafter
adversarial_crafter = CarliniL2Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=x_test.cpu())

# The transformation to tanh space introduce some small perturbation, we remove it to get the exact statistics
adversarial_examples = pd.DataFrame(adversarial_examples)
adversarial_examples[(np.abs(adversarial_examples - x_test_cpu) < 10e-6)] = x_test_cpu
adversarial_examples = adversarial_examples.values

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate(predictions=predictions_adv.long(), real=y_test)
attack = 'CW2'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_CW2.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

### Carlini & Wagner L∞ Attack
*Carlini et al. (2017) "Towards Evaluating the Robustness of Neural Networks"*

In [ ]:
# Creating the adversarial examples crafter
adversarial_crafter = CarliniLInfMethod(classifier,
                                        confidence=0.0,
                                        targeted=False,
                                        learning_rate=0.01,
                                        max_iter=5,
                                        max_halving=5,
                                        max_doubling=5,
                                        eps=0.1,
                                        batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=x_test.cpu())

# The transformation to tanh space introduce some small perturbation, we remove it to get the exact statistics
adversarial_examples = pd.DataFrame(adversarial_examples)
adversarial_examples[(np.abs(adversarial_examples - x_test_cpu) < 10e-6)] = x_test_cpu
adversarial_examples = adversarial_examples.values

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate(predictions=predictions_adv.long(), real=y_test)
attack = 'CW∞'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)
# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_CW∞.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

### Carlini & Wagner L0 Attack
*Carlini et al. (2017) "Towards Evaluating the Robustness of Neural Networks"*

In [ ]:
# Creating the adversarial examples crafter
adversarial_crafter = CarliniL0Method(classifier,
                                      confidence=0.0,
                                      targeted=False,
                                      learning_rate=0.01,
                                      binary_search_steps=10,
                                      max_iter=10,
                                      warm_start=True,
                                      initial_const=0.01,
                                      max_halving=5,
                                      max_doubling=5,
                                      batch_size=128)
# Generating the adversarial examples
adversarial_examples = pd.read_csv("adversarial_examples_CW0.csv").iloc[:, 1:-1].values


adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate(predictions=predictions_adv.long(), real=y_test)
attack = 'CW0'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)
# Exporting the adversarial examples in a .csv file
#pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_CW0.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

### Jacobian-based Saliency Map Attack
*Papernot et al. (2016) The limitations of deep learning in adversarial settings*

In [ ]:
# Creating the adversarial examples crafter
adversarial_crafter = SaliencyMapMethod(classifier,
                                        theta = 0.1,
                                        gamma = 1.0,
                                        batch_size=128)
# Generating the adversarial examples
adversarial_examples = adversarial_crafter.generate(x=x_test.cpu())

adversarial_examples = torch.from_numpy(adversarial_examples).float()
adversarial_examples = adversarial_examples.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
stat_model(model, x_test, y_test, adversarial_examples, y_test)

adversarial_examples_cpu = np.array(adversarial_examples.cpu())
x_test_cpu = np.array(x_test.cpu())

_, predictions_adv = model(adversarial_examples, softmax=True).max(dim=1)
accuracy_adv = evaluate(predictions=predictions_adv.long(), real=y_test)
attack = 'JSMA'
adv_results[attack] = [accuracy_adv] + adv_norms(x_test_cpu, adversarial_examples_cpu)
adv_inv[attack] = adv_criteria(x_test_cpu, adversarial_examples_cpu)

# Exporting the adversarial examples in a .csv file
pd.DataFrame(np.hstack((adversarial_examples_cpu,y_test.cpu().reshape(y_test.shape[0], 1))), columns=data.columns).to_csv("adversarial_examples_JSMA.csv")

# Saving the statistics in a table
perturbation = np.abs(adversarial_examples_cpu - x_test_cpu)
adv_feat_stats[attack] = ((perturbation > 0).sum(axis=0) / perturbation.shape[0]) * 100

print(adv_results[attack])
print(adv_inv[attack])

### Statistics of the different attack methods
The tables below show for every attack:
- The accuracy of the model and mean/max Lp norms between the original and adversarial examples.
- The pourcentage of examples perturbing each feature.
- The proportion of exemples meeting the different invalidation criteria.

In [ ]:
adv_results.to_csv("adv_results.csv")
adv_results

In [ ]:
adv_feat_stats.to_csv("adv_feat_stats.csv")
adv_feat_stats

In [ ]:
fig, ax = plt.subplots(figsize=(7,15))

sb.heatmap(adv_feat_stats.round(decimals=2), annot=True, cmap='YlOrRd', fmt='g')

plt.savefig("hmap_features_stats.eps", format="eps", bbox_inches='tight')
plt.show()

In [ ]:
adv_inv

## Features analysis

### Heat map of the correlation matrix

In [ ]:
numeric_features = ['Duration', 'Packets', 'Bytes', 'Tos', 'Timestamp']
corr = df_training[numeric_features].corr()

fig, ax = plt.subplots(figsize=(5,5))
im = ax.imshow(corr, cmap='RdBu', vmin=-1, vmax=1)

ax.figure.colorbar(im)
plt.yticks(np.arange(0, len(corr.columns), 1), corr.columns)
plt.xticks(np.arange(0, len(corr.columns), 1), corr.columns, rotation=45, ha="right", rotation_mode="anchor")

for i in range(len(numeric_features)):
    for j in range(len(numeric_features)):
        text = ax.text(j, i, corr.iloc[i, j].round(decimals=2), ha="center", va="center", color="w")
        
plt.savefig('correlation_matrix.eps', format='eps', bbox_inches='tight')  
plt.show()